# How To: Convert EMIT .nc to .envi

There are currently 2 similar methods to convert the EMIT netCDF4 files to `.envi` format. Note these only support L1B Radiance, L1B Obs, L2A Reflectance, L2A Reflectance Uncertainty, or L2A Mask to .envi. They do not yet support the L2B Mineral or L2B Mineral Uncertainty products.
1. The `write_envi` function in EMIT tools. This function is still being developed but will currently:
  - Write a GLT output to use for orthocorrection later 
  - Functions from `emit_tools` can be used beforehand to orthorectify if so desired
2. The `reformat.py` script available in the [emit-sds/emit-utils](https://github.com/emit-sds/emit-utils) repository can be used to convert EMIT netCDF files (as delivered to the LP DAAC) to ENVI files. This script also can apply the included GLT to orthorectify the image if desired.

This jupyter notebook walks through how to use both methods to provide users with programmatic routes to accomplish their EMIT reformatting workflows.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  

**Learning Objectives**
+ How to use the `write_envi` function from `emit_tools` module to convert an EMIT netCDF4 to a `.envi` file.
+ How to use the `reformat.py` function from the `emit-utils` repository to convert an EMIT netCDF4 to a `.envi` file.

## Setup

Import packages

In [1]:
import os
import earthaccess

### Authenticate using Earthdata Login and Download the required Granules

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [2]:
earthaccess.login(persist=True)

For this notebook we will download the files necessary using `earthaccess`. You can also access the data in place or stream it, but this can slow due to the file sizes. Provide a URL for an EMIT L2A Reflectance granule.

In [3]:
url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc'

Get an HTTPS Session using your earthdata login, set a local path to save the file, and download the granule asset - This may take a while, the reflectance file is approximately 1.8 GB. 

In [4]:
# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = f'../../data/emit/{granule_asset_id}'
# Download the Granule Asset if it doesn't exist
if not os.path.isfile(fp):
    fs.download(url, fp)

Now lets create an output folder where we will save the `.envi` files.

In [5]:
outpath = '../../data/emit/ag/envi' 
if not os.path.exists(outpath):
    os.makedirs(outpath)

## Method 1: Using `write_envi` from the `emit_tools` module.

Import the necessary packages for this method. 

In [8]:
import sys
sys.path.append('../../tools/emit/python/modules/')
import emit_tools as et

Open the granule using the `emit_xarray` function. We can orthorectify here if so desired.

In [9]:
import emit_tools as et

In [7]:
ds = et.emit_xarray(fp, ortho=True)
ds

ValueError: unrecognized engine h5netcdf must be one of: ['netcdf4', 'scipy', 'pydap', 'rasterio', 'store', 'zarr']

Now, write the dataset as an `.envi` output. If we chose not to orthorectify, you can include a `glt` file to orthorectify later.

In [8]:
et.write_envi(ds, outpath, overwrite=False, extension='.img', interleave='BIL', glt_file=False)

## Method 2: Using reformat.py from emit-utils

### 2.1 Clone and Install emit-utils

Clone the [emit-utils](https://github.com/emit-sds/emit-utils) repository.

In [9]:
!git clone https://github.com/emit-sds/emit-utils.git ../emit_utils/

Cloning into '../emit_utils'...


This will copy the `emit-utils` repository to a folder within this repository. 

After you have copied it, use `pip` package manager to install the directory as a package to ensure you have all of the dependencies and be used in the command line. 

> **This requires that some dependencies already be installed to work properly on Windows. If you have created the Python environment described in the [setup instructions](../../setup/setup_instructions.md) it should work.**

In [10]:
!pip install --editable ../emit_utils

Obtaining file:///C:/Users/ebolch/repos/EMIT-Data-Resources/python/emit_utils
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Running setup.py develop for emit-utils


After successfully installing `emit-utils`, you can use the scripts contained within as part of your workflows. 

### 2.2 Executing the Reformat Script

Before calling the `reformat.py` script, make sure you have an output directory for the `.envi` files that will be produced.

In [11]:
import os
outpath = '../../data/envi' 
if not os.path.exists(outpath):
    os.makedirs(outpath)

Now, execute the `reformat.py` script contained in the emit-utils repository. When executing this script, provide the path to the `.nc` file, followed by the directory to place the `.envi` files in. If you wish to apply the GLT or orthorectify, include `--orthorectify` as an argument.

In [12]:
!python ../emit_utils/emit_utils/reformat.py ../../data/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc ../../data/envi/ --orthorectify

This will orthorectify the image, create an ENVI header, and save it in `.envi` format inside the `../data/envi` folder.

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 07-06-2023  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  